In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from config import weather_api_key
import time
from datetime import datetime
import requests

In [2]:
lats = np.random.uniform(low=-90.000, high=90.000, size=20)
lngs = np.random.uniform(low=-180.000, high=180.000, size=20)
lat_lngs = zip(lats, lngs)
lat_lngs

coordinates = list(lat_lngs)

In [3]:
from citipy import citipy

In [4]:
cities = []

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

20

In [5]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
#print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=cd72838d5ed0a0667f3e2de15d474191


In [6]:
city_data = []


print("Beginning Data Retrieval     ")
print("-----------------------------")


record_count = 1
set_count = 1


# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    
    
    #check if code works 
#print(requests.get(city_url).json())
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})
                            

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


# Run an API request for each of the cities.


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | richards bay
Processing Record 2 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 3 of Set 1 | port alfred
Processing Record 4 of Set 1 | hobart
Processing Record 5 of Set 1 | yumen
Processing Record 6 of Set 1 | jamestown
Processing Record 7 of Set 1 | mahebourg
Processing Record 8 of Set 1 | ponta do sol
Processing Record 9 of Set 1 | angelholm
City not found. Skipping...
Processing Record 10 of Set 1 | mataura
Processing Record 11 of Set 1 | provideniya
Processing Record 12 of Set 1 | korla
Processing Record 13 of Set 1 | kapaa
Processing Record 14 of Set 1 | busselton
Processing Record 15 of Set 1 | sao joao da barra
Processing Record 16 of Set 1 | hami
Processing Record 17 of Set 1 | esperance
Processing Record 18 of Set 1 | wajima
Processing Record 19 of Set 1 | geraldton
Processing Record 20 of Set 1 | grand river south east
City not found. Skipping...
-------------------

In [7]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Richards Bay,-28.7830,32.0377,57.16,76,7,5.44,ZA,clear sky
1,Port Alfred,-33.5906,26.8910,56.75,71,0,15.50,ZA,clear sky
2,Hobart,-42.8794,147.3294,53.46,93,100,4.61,AU,overcast clouds
3,Yumen,40.2833,97.2000,75.58,31,42,8.14,CN,scattered clouds
4,Jamestown,42.0970,-79.2353,77.52,53,20,18.41,US,few clouds
5,Mahebourg,-20.4081,57.7000,68.29,83,40,11.50,MU,scattered clouds
6,Ponta Do Sol,32.6667,-17.1000,86.63,59,77,4.00,PT,broken clouds
7,Mataura,-46.1927,168.8643,44.47,96,100,11.05,NZ,light rain
8,Provideniya,64.3833,-173.3000,41.40,88,100,8.93,RU,overcast clouds
9,Korla,41.7597,86.1469,80.35,29,33,8.68,CN,scattered clouds


In [8]:
new_column_order = ['City', 'Country' ,'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Current Description']
city_data_df = city_data_df[new_column_order]
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Richards Bay,ZA,-28.7830,32.0377,57.16,76,7,5.44,clear sky
1,Port Alfred,ZA,-33.5906,26.8910,56.75,71,0,15.50,clear sky
2,Hobart,AU,-42.8794,147.3294,53.46,93,100,4.61,overcast clouds
3,Yumen,CN,40.2833,97.2000,75.58,31,42,8.14,scattered clouds
4,Jamestown,US,42.0970,-79.2353,77.52,53,20,18.41,few clouds
5,Mahebourg,MU,-20.4081,57.7000,68.29,83,40,11.50,scattered clouds
6,Ponta Do Sol,PT,32.6667,-17.1000,86.63,59,77,4.00,broken clouds
7,Mataura,NZ,-46.1927,168.8643,44.47,96,100,11.05,light rain
8,Provideniya,RU,64.3833,-173.3000,41.40,88,100,8.93,overcast clouds
9,Korla,CN,41.7597,86.1469,80.35,29,33,8.68,scattered clouds


In [9]:
output_data_file = "weather_data/cities.csv"


city_data_df.to_csv(output_data_file, index_label="City_ID")